In [5]:
import sys
sys.path.insert(0, "..")

In [6]:
from utils import access_raster
from glob import glob
import re
import os
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from copy import deepcopy
from scipy.stats import gaussian_kde
import seaborn as sns
import random
from scipy import optimize

In [7]:
model = "withoutmask_all_sgd_adam_1685536886_model_53_01234567"
# model = "1684923533_model_57_012345678"
feature = "HS_diff.tif"
# predfile = "HS_diff.tif"
predfile = "HS_pred.tif"

hs_path = f"/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/{feature}"
modelpredspath = f"/home/abetatos/Escritorio/TFM_Dataset/5.Preds/{model}"

In [8]:
import pandas as pd

def get_slices(aux, precision): 
    df = pd.DataFrame(aux)
    df[0] = df[0].apply(lambda x: round(x, precision))
    arr = []
    for index, round_df in df.groupby(0):
        arr.append([index, round_df[1].mean()])
    return np.array(arr)

In [9]:
for place in os.listdir(hs_path):
    print(place)
    # if place != "HS_Davos_20170316_2m_epsg2056": 
    #     continue
    
    # if place == "snowmap_20160309_ads_100_extent_maskiert_nosnow0_noausreisser":
    #     continue

    feature_raster = access_raster(os.path.join(hs_path, place, f"{feature}.tif"))['array']
    pred_raster = access_raster(os.path.join(modelpredspath, place, predfile))['array']

    fr_flat = feature_raster.flatten()
    pr_flat = pred_raster.flatten()

    aux = [[fr, pr] for fr, pr in zip(fr_flat, pr_flat) if fr not in [0., -9999.] and pr not in [0., -9999.]]
    aux = get_slices(aux, 2)
    random.shuffle(aux)
    fr_flat, pr_flat = np.array(list(zip(*aux[:10000])))
    print(pr_flat.min(), pr_flat.max())
    xy = np.vstack([fr_flat, pr_flat])
    kernel = gaussian_kde(xy)(xy)
    
    fig, ax = plt.subplots()
    ax.scatter(fr_flat, pr_flat, c=kernel, s=1, label="scatter plot")
    # ax.set_ylim((pr_flat.mean()-pr_flat.std()*2, pr_flat.mean()+pr_flat.std()*2))
    ax.set_ylim((min(pr_flat.min(), fr_flat.min()), max(pr_flat.max(), fr_flat.max())))
    ax.set_ylabel(os.path.splitext(predfile)[0])
    ax.set_xlabel(feature)
    plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/HS_diff.tif'